In [27]:
import requests
import pandas as pd

import datetime
import numpy as np 



In [28]:
from datetime import datetime, timedelta

# Khởi tạo ngày giờ bắt đầu và ngày giờ kết thúc
start_datetime = datetime(2024, 1, 15, 0, 0, 0)
end_datetime = datetime(2024, 10, 10, 23,0, 0)

# Danh sách để lưu các nhóm
groups = []

# Biến để giữ ngày giờ hiện tại
current_datetime = start_datetime

while current_datetime <= end_datetime:
    # Tính ngày giờ kết thúc cho nhóm hiện tại
    group_end_datetime = current_datetime + timedelta(days=6, hours=23)
    if group_end_datetime > end_datetime:
        group_end_datetime = end_datetime

    # Lưu nhóm vào danh sách
    groups.append((current_datetime.strftime("%Y-%m-%d %H:%M:%S"), group_end_datetime.strftime("%Y-%m-%d %H:%M:%S")))

    # Cập nhật ngày giờ hiện tại
    current_datetime += timedelta(days=7)
groups

[('2024-01-15 00:00:00', '2024-01-21 23:00:00'),
 ('2024-01-22 00:00:00', '2024-01-28 23:00:00'),
 ('2024-01-29 00:00:00', '2024-02-04 23:00:00'),
 ('2024-02-05 00:00:00', '2024-02-11 23:00:00'),
 ('2024-02-12 00:00:00', '2024-02-18 23:00:00'),
 ('2024-02-19 00:00:00', '2024-02-25 23:00:00'),
 ('2024-02-26 00:00:00', '2024-03-03 23:00:00'),
 ('2024-03-04 00:00:00', '2024-03-10 23:00:00'),
 ('2024-03-11 00:00:00', '2024-03-17 23:00:00'),
 ('2024-03-18 00:00:00', '2024-03-24 23:00:00'),
 ('2024-03-25 00:00:00', '2024-03-31 23:00:00'),
 ('2024-04-01 00:00:00', '2024-04-07 23:00:00'),
 ('2024-04-08 00:00:00', '2024-04-14 23:00:00'),
 ('2024-04-15 00:00:00', '2024-04-21 23:00:00'),
 ('2024-04-22 00:00:00', '2024-04-28 23:00:00'),
 ('2024-04-29 00:00:00', '2024-05-05 23:00:00'),
 ('2024-05-06 00:00:00', '2024-05-12 23:00:00'),
 ('2024-05-13 00:00:00', '2024-05-19 23:00:00'),
 ('2024-05-20 00:00:00', '2024-05-26 23:00:00'),
 ('2024-05-27 00:00:00', '2024-06-02 23:00:00'),
 ('2024-06-03 00:00:

In [29]:
unix_date_ranges = [(int(datetime.strptime(start, '%Y-%m-%d %H:%M:%S').timestamp()), 
                     int(datetime.strptime(end, '%Y-%m-%d %H:%M:%S').timestamp())) 
                    for start, end in groups]

print(unix_date_ranges)

[(1705251600, 1705852800), (1705856400, 1706457600), (1706461200, 1707062400), (1707066000, 1707667200), (1707670800, 1708272000), (1708275600, 1708876800), (1708880400, 1709481600), (1709485200, 1710086400), (1710090000, 1710691200), (1710694800, 1711296000), (1711299600, 1711900800), (1711904400, 1712505600), (1712509200, 1713110400), (1713114000, 1713715200), (1713718800, 1714320000), (1714323600, 1714924800), (1714928400, 1715529600), (1715533200, 1716134400), (1716138000, 1716739200), (1716742800, 1717344000), (1717347600, 1717948800), (1717952400, 1718553600), (1718557200, 1719158400), (1719162000, 1719763200), (1719766800, 1720368000), (1720371600, 1720972800), (1720976400, 1721577600), (1721581200, 1722182400), (1722186000, 1722787200), (1722790800, 1723392000), (1723395600, 1723996800), (1724000400, 1724601600), (1724605200, 1725206400), (1725210000, 1725811200), (1725814800, 1726416000), (1726419600, 1727020800), (1727024400, 1727625600), (1727629200, 1728230400), (1728234000

In [34]:
url_2 =f"https://history.openweathermap.org/data/2.5/history/city?lat=16.0678&lon=108.2208&type=hour&start={start}&end={end}&appid=1d45e07382e657c54d36dc2f4099d1e2"
import json

def parser_weather(json):
    d = dict()  
    d['dt'] = json.get('dt')
    d['temp'] = json.get('main', {}).get('temp')
    d['feels_like'] = json.get('main', {}).get('feels_like')
    d['pressure'] = json.get('main', {}).get('pressure')
    d['humidity'] = json.get('main', {}).get('humidity')
    d['wind_speed'] = json.get('wind', {}).get('speed')
    d['wind_deg'] = json.get('wind', {}).get('deg')
    d['clouds'] = json.get('clouds', {}).get('all')
    weather = json.get('weather', [{}])[0]
    d['weather_main'] = weather.get('main')
    d['weather_description'] = weather.get('description')
    return d
for start, end in unix_date_ranges:

    url = url_2.format(start, end)
    response = requests.get(url)
    data = response.json()
    
    data_list = data['list']


# Sử dụng list comprehension để áp dụng parser_weather cho mỗi item trong data_list
    parsed_data = [parser_weather(item) for item in data_list]

# Tạo DataFrame từ dữ liệu đã parse
    df_weather = pd.DataFrame(parsed_data)

# Chuyển đổi cột 'dt' sang định dạng datetime
    df_weather['dt'] = pd.to_datetime(df_weather['dt'], unit='s').dt.strftime('%Y-%m-%d %H:%M')
    with open('data.csv', 'a', newline='') as file:
        df_weather.to_csv(file, index=False)
   

    

[{'message': 'Count: 120',
  'cod': '200',
  'city_id': 1,
  'calctime': 0.006641179,
  'cnt': 120,
  'list': [{'dt': 1728147600,
    'main': {'temp': 298.14,
     'feels_like': 298.86,
     'pressure': 1013,
     'humidity': 83,
     'temp_min': 298.14,
     'temp_max': 298.14},
    'wind': {'speed': 0.51, 'deg': 0},
    'clouds': {'all': 75},
    'weather': [{'id': 803,
      'main': 'Clouds',
      'description': 'broken clouds',
      'icon': '04n'}]},
   {'dt': 1728151200,
    'main': {'temp': 298.14,
     'feels_like': 298.99,
     'pressure': 1013,
     'humidity': 88,
     'temp_min': 298.14,
     'temp_max': 298.14},
    'wind': {'speed': 1.03, 'deg': 0},
    'clouds': {'all': 75},
    'weather': [{'id': 803,
      'main': 'Clouds',
      'description': 'broken clouds',
      'icon': '04n'}]},
   {'dt': 1728154800,
    'main': {'temp': 298.14,
     'feels_like': 298.86,
     'pressure': 1012,
     'humidity': 83,
     'temp_min': 298.14,
     'temp_max': 298.14},
    'wind': {

TypeError: list indices must be integers or slices, not str

In [20]:
df_weather.describe()

,temp,feels_like,pressure,humidity,wind_speed,wind_deg,clouds
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000
mean,299.920167,302.127000,1012.366667,82.816667,2.361167,122.566667,60.458333
std,1.919199,3.512233,1.371580,9.953660,2.067633,144.381133,21.067676
min,297.140000,298.050000,1009.000000,51.000000,0.000000,0.000000,20.000000
25%,298.140000,299.140000,1011.000000,76.250000,1.030000,0.000000,40.000000
50%,300.020000,302.330000,1012.000000,83.000000,1.540000,28.500000,75.000000
75%,301.140000,304.845000,1014.000000,94.000000,3.555000,292.500000,75.000000
max,305.140000,310.140000,1015.000000,100.000000,9.260000,360.000000,100.000000


In [22]:
with open('data.csv', 'a', newline = '') as file:
    df_weather.to_csv(file, index= False)